In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from operator import attrgetter
import matplotlib.colors as mcolors
import numpy as np

sns.set(rc={'figure.figsize':(10,10)}, style='whitegrid')

###### Таблица с уникальными идентификаторами пользователей

- customer_id — позаказный идентификатор пользователя (аналог номера паспорта)
- customer_unique_id —  уникальный идентификатор пользователя
- customer_zip_code_prefix —  почтовый индекс пользователя
- customer_city —  город доставки пользователя
- customer_state —  штат доставки пользователя

In [2]:
customers = pd.read_csv('/home/jupyter-k-eremina-17/first_project/olist_customers_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter-k-eremina-17/first_project/olist_customers_dataset.csv'

In [ ]:
customers.shape

In [ ]:
customers.value_counts('customer_state')

#### Таблица заказов

- order_id —  уникальный идентификатор заказа (номер чека)
- customer_id —  позаказный идентификатор пользователя (аналог номера паспорта)
- order_status —  статус заказа
- order_purchase_timestamp —  время создания заказа
- order_approved_at —  время подтверждения оплаты заказа
- order_delivered_carrier_date —  время передачи заказа в логистическую службу
- order_delivered_customer_date —  время доставки заказа
- order_estimated_delivery_date —  обещанная дата доставки

In [ ]:
orders = pd.read_csv('/home/jupyter-k-eremina-17/first_project/olist_orders_dataset.csv')

In [ ]:
orders.head()

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

#### Товарные позиции, входящие в заказы

- order_id —  уникальный идентификатор заказа (номер чека)
- order_item_id —  идентификатор товара внутри одного заказа
- product_id —  ид товара (аналог штрихкода)
- seller_id — ид производителя товара
- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- price —  цена за единицу товара
- freight_value —  вес товара


##### Уникальный статусы заказов в таблице olist_order_items_dataset:

- created —  создан
- approved —  подтверждён
- invoiced —  выставлен счёт
- processing —  в процессе сборки заказа
- shipped —  отгружен со склада
- delivered —  доставлен пользователю
- unavailable —  недоступен
- canceled —  отменён

In [ ]:
items = pd.read_csv('/home/jupyter-k-eremina-17/first_project/olist_order_items_dataset.csv')

In [ ]:
items['shipping_limit_date'] = pd.to_datetime(items['shipping_limit_date'])

### Сколько у нас пользователей, которые совершили покупку только один раз? 

In [ ]:
customer_ord = pd.merge(customers, orders, how='left')
customer_ord.head()

In [ ]:
customer_ord.query('order_status == "delivered"') \
    .groupby('customer_unique_id', as_index=False) \
    .agg({'order_id': 'count'}) \
    .value_counts('order_id') \
    .to_frame(name='order_amount').reset_index().query('order_id == 1')

90557 пользователей совершили покупку один раз (~97% всех пользователей).

В расчет берем только те покупки, которые были оплачены и доставлены. 

### Сколько заказов в месяц в среднем не доставляется по разным причинам?

In [ ]:
sns.countplot(customer_ord.query('order_status != "delivered"').order_status)

In [ ]:
orders_not_delivered = customer_ord.query('order_status != "delivered"').drop(columns=['customer_zip_code_prefix', 'customer_city', 'customer_state'])

In [ ]:
orders_not_delivered['order_month'] = orders_not_delivered.order_purchase_timestamp.dt.month_name()

In [ ]:
orders_not_delivered.head()

In [ ]:
new_OND = orders_not_delivered.groupby(['order_month', 'order_status']) \
    .agg({'order_status': 'count'}) \
    .rename(columns={'order_status': 'order_amount'}) \
    .reset_index() \
    .pivot(index='order_month', columns='order_status', values='order_amount')

In [ ]:
new_OND.reset_index(inplace=True)

In [ ]:
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]
new_OND['order_month'] = pd.Categorical(new['order_month'], categories=months, ordered=True)
new_OND.sort_values('order_month', inplace=True)

In [ ]:
new_OND.set_index('order_month', inplace=True)

In [ ]:
new_OND['mean_per_month'] = new_OND.mean(axis=1)

In [ ]:
new_OND['total_per_month'] = new_OND.sum(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(new_OND, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=.5, ax=ax)

In [ ]:
sns.lineplot(data=new_OND['mean_per_month'])

Можно будет отдельно посмотреть на графики причин, pivot через unstack, убрать unavaliable и canselled 

### По каждому товару определить, в какой день недели товар чаще всего покупается.

In [ ]:
items_ord = pd.merge(items, orders, how='left')
items_ord.head()

In [ ]:
items_ord.product_id.nunique()

In [ ]:
items_ord['order_day_of_week'] = items_ord.order_purchase_timestamp.dt.day_name()

In [ ]:
purch_day = items_ord[['product_id', 'order_day_of_week']]

In [ ]:
purchase_day = purch_day.groupby(['product_id', 'order_day_of_week']) \
    .agg({'order_day_of_week': 'count'}) \
    .unstack()

In [ ]:
purchase_day['max_day'] = purchase_day.idxmax(axis=1).apply(lambda x: x[1])

In [ ]:
purchase_day['max_amount'] = purchase_day.max(axis=1)

In [ ]:
result_df = purchase_day[['max_day', 'max_amount']]
result_df

In [ ]:
result_df.value_counts('max_day', normalize=True) * 100

### Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 

In [ ]:
cust_ord = customer_ord.copy() \
    .query('order_status != "unavailable" and order_status != "canceled"') \
    .sort_values('order_purchase_timestamp')

In [ ]:
cust_ord.head()

In [ ]:
cust_ord['order_month'] = cust_ord.order_purchase_timestamp.dt.strftime('%Y-%b')

In [ ]:
avg_month = cust_ord.groupby(['order_month', 'customer_unique_id'], as_index=False).agg({'order_id': 'count'})

In [ ]:
avg_month.order_month = pd.to_datetime(avg_month.order_month)

In [ ]:
avg_month['day_in_month'] = avg_month.order_month.dt.daysinmonth

In [ ]:
avg_month['result'] = avg_month. order_id / (avg_month.day_in_month / 7)

In [ ]:
avg_month.value_counts('result')

### Функция, позволяющая строить когортный анализ.

In [ ]:
cohort = customer_ord.copy()

In [ ]:
cohort.groupby('customer_unique_id')['order_purchase_timestamp'].min()

In [ ]:
n_orders = cohort.groupby(['customer_unique_id'])['order_id'].nunique()
mult_orders_perc = np.sum(n_orders > 1) / cohort['customer_unique_id'].nunique()
print(f'{100 * mult_orders_perc:.2f}% of customers ordered more than once.')

In [ ]:
ax = sns.distplot(n_orders, kde=False, hist=True)
ax.set(title='Distribution of number of orders per customer',
       xlabel='# of orders', 
       ylabel='# of customers');

In [ ]:
coh = cohort[['customer_unique_id', 'order_id', 'order_purchase_timestamp']]
coh.head()

In [ ]:
coh['order_month'] = coh['order_purchase_timestamp'].dt.to_period('M')

In [ ]:
coh['cohort'] = coh.groupby('customer_unique_id')['order_purchase_timestamp'] \
                 .transform('min') \
                 .dt.to_period('M')

In [ ]:
df_cohort = coh.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('customer_unique_id', 'nunique')) \
              .reset_index(drop=False)

In [ ]:
df_cohort['period_number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))

In [ ]:
df_cohort.head()

In [ ]:
df_cohort.reset_index(inplace=True)
df_cohort.set_index(['period_number', 'cohort'], inplace=True)

cohort_group_size = df_cohort['n_customers'].groupby(level=1).first()
cohort_group_size.head()

In [ ]:
user_retention = df_cohort['n_customers'].unstack(1).T.divide(cohort_group_size, axis=0).drop(columns=0)
user_retention.head()

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(user_retention, 
                mask=user_retention.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('Monthly Cohorts: User Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_group_size = pd.DataFrame(cohort_group_size).rename(columns={0: 'cohort_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_group_size, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])

    fig.tight_layout()

### RFM-кластеры для пользователей.

In [ ]:
rmf = items_ord.copy()

In [ ]:
# узнаем дату самого старого заказа
last_date = rmf['order_delivered_carrier_date'].max()

Формируем метрики, где:
 - recency - количество дней от момента первого заказа
 - frequency - частота заказов
 - monetary_value - общая сумма трат на пользователя

In [ ]:
rfm_table = rmf.groupby('customer_id') \
    .agg({'order_delivered_carrier_date': lambda x: (last_date - x.max()).days,
            'order_id': lambda x: len(x), 
            'price': lambda x: x.sum()})

In [ ]:
rfm_table.rename(columns={'order_delivered_carrier_date': 'recency', 
                          'order_id': 'frequency', 
                          'price': 'monetary_value'}, inplace=True)

In [ ]:
# делим на группы по квантилям  
quantiles = rfm_table.quantile(q=[0.3, 0.6]).to_dict()

In [ ]:
# приславаем оценку с разбивкой по квантилю
def RScore(x,p,d):
     if x <= d[p][0.3]:
         return 1
     elif x <= d[p][0.6]:
         return 2
     else:
         return 3

In [ ]:
segmented_rfm = rfm_table.copy()

In [ ]:
segmented_rfm['r_quantile'] = segmented_rfm['recency'].apply(RScore, args=('recency', quantiles))
segmented_rfm['f_quantile'] = segmented_rfm['frequency'].apply(RScore, args=('frequency', quantiles))
segmented_rfm['m_quantile'] = segmented_rfm['monetary_value'].apply(RScore, args=('monetary_value', quantiles))

In [ ]:
# собираем RFM рейтинг
segmented_rfm['RFMScore'] = segmented_rfm.r_quantile.map(str) + segmented_rfm.f_quantile.map(str) + segmented_rfm.m_quantile.map(str)

In [ ]:
segmented_rfm.head()

In [ ]:
# считаем среднее по каждому кластеру
mean_cluster = segmented_rfm.groupby('RFMScore', as_index=False) \
    .agg({'recency': 'mean',
        'frequency': 'mean',
        'monetary_value': 'mean'}) \
    .sort_values('RFMScore')